# **Анализ данных.**

Для начала, я просмотрел исходные данные по диогонали. Буквально в текстовом редакторе. Обнаружил, что поле Model имеет большое количество неточностей ввода. Например ILX5-Speed это модель Acura ILX с пятиступенчатой коробкой передач, причем неясно какой именно - автомат или механика. Плюс ILX5-Speed и ilx 5 - speed создадут больше категорий, чем есть на самом деле. Поэтому я решил при помощи регулярных выражений привести State, City, Model, Make к общему виду. То есть только символы и цифры без разделителей и пробелов, в верхнем регистре. 

Что мы имеем до обработки:<br>
Make - 57 категорий, Model - 2694, State - 58, City - 2552. Особенно развеселил State.<br>
После обработки:<br>
Make - 57 категорий, Model - 2628, State - 50, City - 2388.<br>
Попробую оценить, насколько эта манипуляция влияет на что-либо. В качестве критерия я выбрал $R^2$ оценку XGBoost. Предсказывать буду цену. На данный момент не суть важно, в каком состоянии данные вобще, я хочу получить объективную относительную оценку на одном и том-же наборе данных.

До обработки 0.6638698701826335 и 0.6607462948411851 после. Точность как-бы даже упала. Но я всё равно решил оставить эту обработку, поскольку она логически обоснована.

![title](img/01.png)![title](img/02.png)


Теперь попробую проверить то-же самое, но на "чистом" датасете. То есть, не нулевые цена и пробег, убраны строки со значениями NaN. Все условия выполняются одновременно.

Получилось на 228650 строк меньше. Не здорово, но 623472 строки на мой взгляд всё равно позволяют хорошо обучить модель. 

До обработки 0.8720984096737733 и 0.8702714715047611 после. Разница не велика, но она есть. На данный момент могу предположить, что либо значисомть предикторов невелика, либо уменьшение категорий не значимо в принципе.

![title](img/03.png)![title](img/04.png)

А вот это очень интересно. На "чистом" датасете с обработанными категориями выросла скорость обучения! С 821 до 539 итераций. Я пока "плаваю" в детальном понимании XGBoost, но скорость обучения существенно выросла однозначно. Это плюс. Любопытно каким образом город более значим для регрессии, чем год выпуска. Попробую разобраться.

![title](img/05.png)

Тут ничего неожиданного.

Все 1792 дубликата были удалены в самом начале.

![title](img/06.png)

Ну чего-то такого я и ожидал. Но работать с этим датасетом дальше имеет смысл только после импутации данных.

# **Импутация данных**

### **Mileage**

Для нулевых значений нужно проверить год. Если год максимальный, то это новая машина. Можно смело вместо нуля поставить, например, 5. Поскольку автомобиль ездил по заводу, салону и тому подобное.

Всего 135 записей.

Порченых значений пробега 121К. Я обучил регрессию для предсказания тех значений, которые не пересекаются с порченной ценой и моделью. Это 103K записей. Точность составила 82%. Я считаю, это точнее, чем заполнить средним или просто каким-то конкретным значением.